### データ読み込み

In [3]:
import pandas as pd

df = pd.read_csv("sample_data.csv")
print(df.dtypes)
df

No        int64
Weight    int64
Value     int64
dtype: object


,No,Weight,Value
0,1,90,108
1,2,120,96
2,3,120,144
3,4,210,168
4,5,160,192
5,6,180,144
6,7,280,336
7,8,40,32
8,9,80,96
9,10,50,40


### ナップサック問題

In [4]:
from ortools.linear_solver import pywraplp

In [5]:
### ソルバー処理用に辞書に必要な情報を格納
def create_data_model(df, num_bins, limit_item_capacity):
    """サンプルデータの作成
    　df: 読み込むDataFrame、num_bins 設定する bin の数、limit_item_capacity: bin の中にいれられる item の数の上限
    """
    
    data = {}
    weights = df["Weight"].values.tolist() # 重量
    values = df["Value"].values.tolist() # 評価価値

    data['weights'] = weights
    data['values'] = values

    data['items'] = list(range(len(weights))) # item No
    data['num_items'] = len(weights)
    num_bins = num_bins
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [1000] * num_bins
    data['max_items'] = [limit_item_capacity] * len(weights)
    return data

In [6]:
# SCIP でソルバーを作成 
solver = pywraplp.Solver.CreateSolver('SCIP')

In [7]:
data = create_data_model(df, 10, 3)

In [8]:
# 変数
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

In [9]:
# 制約条件
# 全ての item は bin のどこかに 1つだけ入る
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) <= 1)

# bin_capacities で設定した重量以上の item を格納しない
for j in data['bins']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i]
            for i in data['items']) <= data['bin_capacities'][j])
    
# max_items に設定した数以上の item 数を格納しない
for j in data['bins']:
    bin_count = 0
    for i in data['items']:
        bin_count += x[i, j]
    solver.Add(bin_count <= data['max_items'][j])

In [10]:
# 目的変数の設定
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        objective.SetCoefficient(x[(i, j)], data['values'][i])
objective.SetMaximization()

status = solver.Solve()

In [11]:
df_knapsack = pd.DataFrame() # 集約するための空のDataFrame

if status == pywraplp.Solver.OPTIMAL:
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0
        for i in data['items']:
            if x[i, j].solution_value() > 0:
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]

                # 必要な情報を 一時的な DataFrameに格納して df_knapsack に結合
                df_tmp = pd.DataFrame(
                    [
                        j, # Bin
                        i, # item
                        data['weights'][i], # weight
                        data['values'][i], # values
                    ]
                ).T
                df_tmp.columns=["bin", "item", "weight", "value"]
                df_knapsack = pd.concat([df_knapsack, df_tmp], axis=0)

        if bin_weight == 0: # 何も入らない bin が登場したら break で処理を終わらせる
            break
        total_weight += bin_weight
else:
    print('The problem does not have an optimal solution.')

df_knapsack = df_knapsack.reset_index(drop=True)
assert df_knapsack["weight"].sum() == df["Weight"].sum() # 元データとナップサック処理後のデータで重量が一致するかテスト
display(df_knapsack)

,bin,item,weight,value
0,0,0,90,108
1,0,1,120,96
2,0,2,120,144
3,1,3,210,168
4,1,4,160,192
5,1,5,180,144
6,2,6,280,336
7,2,7,40,32
8,2,8,80,96
9,3,9,50,40
